# Imports

In [1]:
from ibm_watson_machine_learning import APIClient

# 

# Credenciales

In [2]:
#Credenciales de acceso al deploy

wml_credentials = {
    "apikey":"API_KEY_VALUE",
    "url":"https://us-south.ml.cloud.ibm.com"
}


# 

# Funcion a  deployar

In [3]:
def deployable_callable():
    """
    Función de python deployable que predice la persona del audio e identifica si es el locutor o un impostor
    """
    
    #Primero hacemos imports y seteamos configuracion inicial
    
    import json
    from google.cloud import storage
    from google.oauth2 import service_account

    from botocore.client import Config
    import ibm_boto3
    import pickle
    from io import BytesIO

    import librosa
    import numpy as np
    
    import pandas as pd
    
    
    
    
    ibm_cos_client = ibm_boto3.client(service_name='s3',
        ibm_api_key_id='IBM_API_KEY',
        ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
        config=Config(signature_version='oauth'),
        endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')
        
    
    
    
    ##############################################################################################################################
    ##############################################################################################################################
    ##############################################################################################################################
    
    
    def get_blob_to_predict_and_bucket(filename):
        """
            Funcion que recibe como parámetro el nombre del audio que hay que predecir, y retorna el blob del mismo
        """
    
        #Definimos las credenciales
        credentials = {
        "type": "service_account",
        "project_id": "cuidartech-7e54f",
        "private_key_id": "PRIVATE_KEY_ID",
        "private_key": "PRIVATE_KEY",
        "client_email": "firebase-adminsdk-8qlnh@cuidartech-7e54f.iam.gserviceaccount.com",
        "client_id": "108378739182204427784",
        "auth_uri": "https://accounts.google.com/o/oauth2/auth",
        "token_uri": "https://oauth2.googleapis.com/token",
        "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
        "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/firebase-adminsdk-8qlnh%40cuidartech-7e54f.iam.gserviceaccount.com"
        }


        #Cargamos las credenciales como un json, y nos conectamos al cliente del storage de Firebase
        json_credentials = json.loads(json.dumps(credentials))

        credentials = service_account.Credentials.from_service_account_info(json_credentials)

        storage_client = storage.Client(credentials=credentials)


        #Creamos la instancia del bucket en donde tenemos los archivos y luego obtenemos el blob solicitado

        bucket = storage_client.get_bucket("cuidartech-7e54f.appspot.com")

        blob = list(bucket.list_blobs(prefix=filename))[0]

        return blob, bucket


    ##############################################################################################################################
    ##############################################################################################################################
    ##############################################################################################################################    
    
    
    def load_model_from_user(username, boto3_client):
        """
            Función que carga el modelo entrenado de un determinado usuario
        """

        username = username.replace(".", "_")

        model_filename = str.upper(username)+".pickle"

        streaming_body_1 = ibm_cos_client.get_object(Bucket='pruebasgian-donotdelete-pr-3aeebt0ewfpjpi', 
                                                                              Key=model_filename)['Body']

        model = pickle.load(BytesIO(streaming_body_1.read()))

        return model
    
    
    ##############################################################################################################################
    ##############################################################################################################################
    ##############################################################################################################################    
        
    
    def load_MFCC_archivo(archivoVoz, sr=16000, n_mfcc=16, n_fft=512, hop_length=256):
        """
        Función que procesa un archivo para obtener los coeficientes utilizados en la prediccion
        Carga de los coeficientes Cepstrum en la escal Mel de un archivo utilizando la librería librosa.
        Carga de los coeficientes Cepstrum delta obteniendo información temporal y fonética.




        Params:

        archivoVoz = archivo a procesar
        sr = Sampling Rate del archivo (en Hz)
        n_mfcc = cantidad de coeficientes cepstrales a extraer
        n_fft = tamaño de la ventana de la transformada de Fourier
        hop_length = tamaño del salto para el procesamiento en frames



        Para mas información:

        MFCC = https://es.wikipedia.org/wiki/MFCC

        Librosa MFCC = https://librosa.org/doc/main/generated/librosa.feature.mfcc.html
        """

        y,sr=librosa.load(archivoVoz, sr=sr)

        mfcc = librosa.feature.mfcc(y, sr=sr, n_mfcc=n_mfcc, dct_type=2, norm='ortho',
                                    hop_length=hop_length, n_fft=n_fft) 


        #Actualmente en el array tengo n filas x m columnas, donde n es la cantidad de MFCC y m la cantidad de frames
        #Traspongo la matriz para quedarme con los features de un determinado frame, es decir n cantidad de MFCC
        #columnas x m cantidad de frames filas
        mfcc = np.transpose(mfcc)


        #Para agregar información temporal y fonética (velocidad, aceleración, entre otras) a los coeficientes
        #agrego los delta de cada feature y los concateno al array en formato de columnas
        mfcc = np.append(mfcc, librosa.feature.delta(mfcc), axis = 1)



        return mfcc
    
        
    ##############################################################################################################################
    ##############################################################################################################################
    ##############################################################################################################################           
    
    
    def make_predictions(model, mfccs):
        """
            Función que utiliza un modelo entrenado y predice utilizando los mfcc de todos los frames de un audio.
            Se realiza una predicción para cada frame
        """

        return model.predict(mfccs)
    
       
    ##############################################################################################################################
    ##############################################################################################################################
    ##############################################################################################################################     
    
    
    def load_names_of_trained_model_from_user(username, boto3_client):
        """
            Función que carga los nombres de las personas del modelo entrenado de un determinado usuario
        """

        username = username.replace(".", "_")

        filename = str.upper(username)+"_NAMES.pickle"

        streaming_body_1 = ibm_cos_client.get_object(Bucket='pruebasgian-donotdelete-pr-3aeebt0ewfpjpi', 
                                                                              Key=filename)['Body']

        names = pickle.load(BytesIO(streaming_body_1.read()))

        return names
        
        
    ##############################################################################################################################
    ##############################################################################################################################
    ##############################################################################################################################    
    
    
    def prepare_features_to_validate_prediction(predictions, names):
        """
        Función que recibe las predicciones y los nombres de las personas que se entrenaron en el modelo.
        Devuelve el nombre de la persona predicha, el porcentaje asignado de frames sobre el total del audio, 
        y el desvío de frames asignados a la primera y segunda persona.


        - Nombre de la persona predicha, para saber si la predicción es correcta.
        - Porcentaje asignado de frames sobre el total del audio, para saber si asigna la mayoría a la persona predicha.
        - Desvío de frames asignados a la primera y segunda persona, para saber que tan confiable es la predicción.

        """


        total_frames_audio = len(predictions)

        #Total de frames predichos para una persona
        total_predictions = [0,0,0,0,0,0,0,0]

        for pred in predictions:
            total_predictions[pred.argmax()]+=1


        #Creación del df con las predicciones en frames por persona
        df = pd.DataFrame({"nombres_personas": names, "predicciones": total_predictions}
                         ).sort_values("predicciones", ascending=False).reset_index(drop=True)


        #Persona predicha
        persona_predicha = df['nombres_personas'][0]

        #Frames persona predicha
        frames_persona_predicha = df['predicciones'][0]

        #Porcentaje de frames predichos sobre el total del audio
        porcentaje_frames_persona_predicha_sobre_total = (frames_persona_predicha/total_frames_audio)*100

        #Desvio de asignación de frames entre la persona a la que más frames le asigna y la segunda.
        #Utilizado para saber qué tan seguro se está de la predicción.
        desvio_frames_primer_y_segunda_persona = (frames_persona_predicha/df['predicciones'][1])-1


        return persona_predicha, porcentaje_frames_persona_predicha_sobre_total, desvio_frames_primer_y_segunda_persona, df
        
        
    ##############################################################################################################################
    ##############################################################################################################################
    ##############################################################################################################################      
     
        
    def validar_prediccion(porcentaje_sobre_total_audio, desvio_frames_persona_predicha_y_segunda):
        """
            Función que recibe el porcentaje de frames asignados a la persona predicha sobre el total del audio, y
            el porcentaje de desvio de asignacion de frames entre la persona predicha y la segunda con más frames asignados.

            Devuelve si la predicción es confiable.

            La predicción es confiable, si el total de frames asignado es mayor al 40% del audio y si hay una diferencia de un 50% entre
            la persona predicha y la segunda.
        """

        validacion_desvio = (1 if desvio_frames_persona_predicha_y_segunda>=.5 else 
                             (.5 if desvio_frames_persona_predicha_y_segunda>=.25 else 0))
        validacion_porcentaje = (1 if np.ceil(porcentaje_sobre_total_audio)>=40 else 0)

        return min(validacion_desvio, validacion_porcentaje)
    
        
    ##############################################################################################################################
    ##############################################################################################################################
    ##############################################################################################################################      
               

    def score(payload):
        """
            Predict and validate prediction
        
        """
        


        try:
        
            #Obtener el nombre del audio de la API call, y su blob.
            
            audio_to_predict_name = payload['input_data'][0]['values']
            
            blob_audio, bucket = get_blob_to_predict_and_bucket(audio_to_predict_name)

            
            
            #Cargar el modelo del usuario 
            
            user_expected = blob_audio.name.split('/')[1].upper() 
            
            model = load_model_from_user(user_expected, ibm_cos_client)
                        
            
            
            #Carga del archivo del audio y procesado para extracción de MFCC
            file = open('temp_audio.wav', 'wb+')
            bucket.get_blob(blob_audio.name).download_to_file(file)
            file.close()

            mfcc = load_MFCC_archivo(file.name)

            
            
            #Predicción sobre los MFCC
            predictions = make_predictions(model, mfcc)
            
            
            
            #Obtención de los nombres de las personas con las que se entrenó el modelo           
            nombres_personas = load_names_of_trained_model_from_user(user_expected, ibm_cos_client)
            
            
            
            #Obtención de los features para validar la predicción
            predicted_person, porcentaje_frames_sobre_total_audio, desvio_asignacion_frames, df = prepare_features_to_validate_prediction(predictions, nombres_personas)

            
            #Validación de la predicción
            validacion = validar_prediccion(porcentaje_frames_sobre_total_audio, desvio_asignacion_frames)
            
            
            #Procesado de la validación
            return_message = ""

            
            if validacion==1:
                
                if user_expected!=predicted_person:
                    return_message = "Validación incorrecta para "+user_expected.replace(".", " ")+". Se predijo que su nombre es "+predicted_person+". Usted es un impostor!!!"
                    
                else:    
                    return_message = "Validación correcta para "+user_expected.replace(".", " ")+"!!!"

            elif ((validacion > 0) and (user_expected==predicted_person)) or ((validacion==0) and (user_expected==predicted_person)):
                return_message = "Validación dudosa para "+user_expected.replace(".", " ")+". Por favor, repita la prueba!!!"

            else:
                return_message = "Validación incorrecta para "+user_expected.replace(".", " ")+". Usted es un impostor!!!"
                

            
            return {'predictions': [{'values': return_message}]}
            
        
        
        except Exception as e:
            return {'predictions': [{'values': "Error when trying to make a prediction for the audio"+audio_to_predict_name+". Please check the logs and try in a few moments!"}]}
        
    return score

# 

# 

# 

# Comenzamos configuración para deploy y deploy

# 

# Seteamos el cliente y el espacio al que deployamos

In [ ]:
#creamos el cliente de ML

wml_client = APIClient(wml_credentials)

wml_client.spaces.list()


In [6]:
#Guardamos el ID del espacio donde se desplegará el modelo y lo seteamos como default

SPACE_ID = "SPACE_ID_TO_DEPLOY_FROM_wml_client.spaces.list()"

wml_client.set.default_space(SPACE_ID)

'SUCCESS'

# 

# Configuramos el entorno al que deployamos

In [7]:
wml_client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

# 

### Creamos el yaml con las librerias necesarias extras, seteamos el software spec base, y agregamos las librerias del yaml

In [8]:
config_yml =\
"""name: python38
channels:
  - conda-forge    
  - nodefaults
dependencies:
  - librosa
  - pip:
    - google-cloud-storage

prefix: /opt/anaconda3/envs/python38
"""

with open("config.yaml", "w", encoding="utf-8") as f:
    f.write(config_yml)

In [9]:
base_sw_spec_uid = wml_client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
base_sw_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [10]:
!cat config.yaml

name: python38
channels:
  - conda-forge    
  - nodefaults
dependencies:
  - librosa
  - pip:
    - google-cloud-storage

prefix: /opt/anaconda3/envs/python38


el YAML tiene el detalle de librerías adicionales como un package. Ahora desplegamos el package para poder usarlo

In [11]:
meta_prop_pkg_extn = {
    wml_client.package_extensions.ConfigurationMetaNames.NAME: "Librosa XXS env (deploy)",
    wml_client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Environment with Librosa and libraries to make predictions",
    wml_client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml"
}

pkg_extn_details = wml_client.package_extensions.store(meta_props=meta_prop_pkg_extn, file_path="config.yaml")
pkg_extn_uid = wml_client.package_extensions.get_uid(pkg_extn_details)
pkg_extn_url = wml_client.package_extensions.get_href(pkg_extn_details)

Creating package extensions
SUCCESS


# 

### Desplegamos el software SPEC sobre el que se ejecutará la función y agregamos el package con las librerias

In [12]:
meta_prop_sw_spec = {
    wml_client.software_specifications.ConfigurationMetaNames.NAME: "Librosa XXS (deploy) software_spec",
    wml_client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": base_sw_spec_uid}
}

sw_spec_details = wml_client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_uid = wml_client.software_specifications.get_uid(sw_spec_details)

wml_client.software_specifications.add_package_extension(sw_spec_uid, pkg_extn_uid)

SUCCESS


'SUCCESS'

# 

### Guardamos la función de predicción

In [13]:
meta_props = {
    wml_client.repository.FunctionMetaNames.NAME: "Make prediction function",
    wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid
}

function_details = wml_client.repository.store_function(meta_props=meta_props, function=deployable_callable)
function_uid = wml_client.repository.get_function_uid(function_details)

In [14]:
wml_client.repository.get_details(function_uid)

{'entity': {'software_spec': {'id': '4ea98c00-d77d-4bef-852c-d6e44f7e9791',
   'name': 'Librosa XXS (deploy) software_spec'},
  'type': 'python'},
 'metadata': {'created_at': '2022-10-25T00:27:22.770Z',
  'id': '81882741-f015-4ed2-be37-ec07d104d9a2',
  'modified_at': '2022-10-25T00:27:25.093Z',
  'name': 'Make prediction function',
  'owner': 'IBMid-6650007T1K',
  'space_id': '8a50bf52-17ff-401d-a128-7502ce8436f4'},
 'system': {'warnings': []}}

In [15]:
wml_client.repository.list_functions()

------------------------------------  ------------------------  ------------------------  ------
GUID                                  NAME                      CREATED                   TYPE
81882741-f015-4ed2-be37-ec07d104d9a2  Make prediction function  2022-10-25T00:27:22.002Z  python
c589d1c3-a54d-42d4-9572-7206ae6b8e4d  Make prediction function  2022-10-24T23:42:43.002Z  python
358f33e0-fa8e-40fb-a616-770cd19efb50  Make prediction function  2022-10-21T02:14:23.002Z  python
6d2ede50-785f-4bb1-9357-9dcea4226a7d  Train model function      2022-10-15T22:02:17.002Z  python
fb5b9ce5-55ca-4392-b21c-d81a49847ab6  Train model function      2022-10-15T22:00:25.002Z  python
6d28a0c3-af01-45db-963f-f73e31e54527  Train model function      2022-10-15T21:34:22.002Z  python
------------------------------------  ------------------------  ------------------------  ------


# 

# Desplegamos la función

In [16]:
metadata = {
    wml_client.deployments.ConfigurationMetaNames.NAME: "Deployment of Make prediction function",
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

function_deployment = wml_client.deployments.create(function_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: '81882741-f015-4ed2-be37-ec07d104d9a2' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
............................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='9d7b9686-4e69-432b-a43f-0a7a8aa097ce'
------------------------------------------------------------------------------------------------




In [17]:
wml_client.deployments.list()

------------------------------------  --------------------------------------  -----  ------------------------
GUID                                  NAME                                    STATE  CREATED
9d7b9686-4e69-432b-a43f-0a7a8aa097ce  Deployment of Make prediction function  ready  2022-10-25T00:27:27.019Z
2cf02cf2-3bd6-4dd9-9b72-0a1c2f5cd1fd  Deployment of Make prediction function  ready  2022-10-24T23:42:48.294Z
93cfc89d-8bf1-4079-8cca-88e4718f9bff  Deployment of Make prediction function  ready  2022-10-21T02:16:32.909Z
b46d225b-7b39-42fb-a219-e6bb74c7464f  Deployment of Train model function      ready  2022-10-15T22:02:23.437Z
------------------------------------  --------------------------------------  -----  ------------------------


In [18]:
deployment_id = wml_client.deployments.get_uid(function_deployment)

deployment_id

'9d7b9686-4e69-432b-a43f-0a7a8aa097ce'

# 

# 

# 

# 

# Testeo de la función desplegada

In [19]:
scoring_payload = {
    "input_data": [{
        'values': "audios/Gian.grillo/1666222906813.MPEG4"
    }]
}

In [20]:
%%time
predictions = wml_client.deployments.score(deployment_id, scoring_payload)
print(predictions["predictions"][0]["values"])

Validación correcta para GIAN GRILLO!!!
CPU times: user 6.75 ms, sys: 7.36 ms, total: 14.1 ms
Wall time: 7.17 s
